In [1]:
import healpy as hp
import numpy as np
import os
from lsst.daf.persistence import Butler
repo_dir = '/datasets/hsc/repo/rerun/DM-13666/DEEP'
butler = Butler(repo_dir)
#/datasets/hsc/repo/rerun/DM-13666/DEEP/deepCoadd-results/HSC-R

In [2]:
nside = 512
npix = hp.nside2npix(nside)
print('resolution %e sq degrees' % (4.0*np.pi*(180.0/np.pi)**2/npix))

resolution 1.311396e-02 sq degrees


In [3]:
pixel_counts = np.zeros(npix, dtype=float)

In [4]:
sky_map = butler.get('deepCoadd_skyMap')

In [5]:
catalog_name = 'deepCoadd_forced_src'
ref_name = 'deepCoadd_ref'
filter_name = 'HSC-R'

In [6]:
tract_root = os.path.join(repo_dir, 'deepCoadd-results', filter_name)
tract_list = os.listdir(tract_root)
tract_list = np.array(tract_list).astype(int)

In [9]:
valid_data_id = []
for tract_id in tract_list:
    tract = sky_map[tract_id]
    patches = tract.getNumPatches()
    for ip1 in range(patches[0]):
        for ip2 in range(patches[1]):
            dataId = {'filter':filter_name,
                      'tract':tract_id,
                      'patch':'%d,%d' % (ip1, ip2)}
            if not butler.datasetExists(catalog_name, dataId=dataId):
                continue
            else:
                valid_data_id.append(dataId)

In [12]:
ref_cat = butler.get(ref_name, dataId=valid_data_id[0])
src_cat = butler.get(catalog_name, dataId=valid_data_id[0])

In [14]:
len(ref_cat)

23340

In [15]:
len(src_cat)

23340

In [17]:
np.testing.assert_array_equal(ref_cat['coord_ra'], src_cat['coord_ra'])

In [18]:
np.testing.assert_array_equal(ref_cat['coord_dec'], src_cat['coord_dec'])

In [19]:
ref_cat['detect_isPrimary']

array([False, False, False, ..., False, False, False])

In [43]:
ref_cat.schema.find('base_PsfFlux_flux')

SchemaItem(key=Key<D>(offset=896, nElements=1), field=Field['D'](name="base_PsfFlux_flux", doc="flux derived from linear least-squares fit of PSF model", units="count"))

In [44]:
#src_cat.schema.getNames()

In [ ]:
src_val = src_cat['base_ClassificationExtendedness_value']
ref_val = ref_cat['base_ClassificationExtendedness_value']
flux_flag = ref_cat['modelfit_CModel_flag']
src_psf_flux = src_cat['base_PsfFlux_flux']
src_model_flux = src_cat['modelfit_CModel_flux']
ref_psf_flux = ref_cat['base_PsfFlux_flux']
ref_model_flux =ref_cat['modelfit_CModel_flux']

src_ratio = src_psf_flux/src_model_flux
ref_ratio = ref_psf_flux/ref_model_flux
for ii in range(len(src_cat)):
    if src_val[ii] != ref_val[ii] and not np.isnan(src_val[ii]) and \
    not np.isnan(ref_val[ii]):
        print(src_val[ii], ref_val[ii],
              src_ratio[ii], ref_ratio[ii])

In [46]:
help(hp.ang2pix)

Help on function ang2pix in module healpy.pixelfunc:

ang2pix(nside, theta, phi, nest=False, lonlat=False)
    ang2pix : nside,theta[rad],phi[rad],nest=False,lonlat=False -> ipix (default:RING)
    
    Parameters
    ----------
    nside : int, scalar or array-like
      The healpix nside parameter, must be a power of 2, less than 2**30
    theta, phi : float, scalars or array-like
      Angular coordinates of a point on the sphere
    nest : bool, optional
      if True, assume NESTED pixel ordering, otherwise, RING pixel ordering
    lonlat : bool
      If True, input angles are assumed to be longitude and latitude in degree,
      otherwise, they are co-latitude and longitude in radians.
    
    Returns
    -------
    pix : int or array of int
      The healpix pixel numbers. Scalar if all input are scalar, array otherwise.
      Usual numpy broadcasting rules apply.
    
    See Also
    --------
    pix2ang, pix2vec, vec2pix
    
    Examples
    --------
    >>> import healpy 

In [ ]:
ct = 0
for dataId in valid_data_id:
    ref_cat = butler.get(ref_name, dataId=dataId)
    is_primary = ref_cat['detect_isPrimary']
    is_star = ref_cat['base_ClassificationExtendedness_value'] < 0.99
    valid = is_primary & is_star
    data = ref_cat[valid]
    pixel_number = hp.ang2pix(nside,
                              np.degrees(data['coord_ra']),
                              np.degrees(data['coord_dec']),
                              lonlat=True)
    pixel_counts[pixel_number]+=1
    ct += 1
    if ct %10 ==0:
        print('ran %d of %d dataIds' % (ct, len(valid_data_id)))

ran 10 of 1305 dataIds
ran 20 of 1305 dataIds
ran 30 of 1305 dataIds
ran 40 of 1305 dataIds
ran 50 of 1305 dataIds
ran 60 of 1305 dataIds
ran 70 of 1305 dataIds
ran 80 of 1305 dataIds
ran 90 of 1305 dataIds
ran 100 of 1305 dataIds
ran 110 of 1305 dataIds
ran 120 of 1305 dataIds
ran 130 of 1305 dataIds
ran 140 of 1305 dataIds
ran 150 of 1305 dataIds
ran 160 of 1305 dataIds
ran 170 of 1305 dataIds
ran 180 of 1305 dataIds
ran 190 of 1305 dataIds
ran 200 of 1305 dataIds
